In [1]:
import glob
import numpy
from PIL import Image
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from datetime import date
from datetime import datetime
import cv2

Using TensorFlow backend.


In [2]:
import tensorflow as tf
import numpy as np
import argparse
import math
import os
from matplotlib.image import imread
from scipy.misc.pilutil import imresize, imsave
import matplotlib.pyplot as plt

In [3]:
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Reshape, Flatten, Dropout, Input
from keras.layers.convolutional import UpSampling2D, Conv2D, Conv2DTranspose, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam, SGD
from keras import initializers

import numpy as np
import random

%matplotlib inline
import matplotlib.pyplot as plt

In [4]:
def build_generator():
    
        model = Sequential()
        model.add(Dense(128 * 16 * 8, input_dim = latent_dim))
        model.add(BatchNormalization())
        model.add(LeakyReLU())
        model.add(Reshape((8, 8, 256)))
        
        model.add(Conv2DTranspose(1024, filter_size_g, strides=(1,1), padding='same'))
        #model.add(Conv2D(512, filter_size_g, strides=(1,1), padding='same'))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU())
        
        model.add(Conv2DTranspose(512, filter_size_g, strides=(1,1), padding='same'))
        #model.add(Conv2D(512, filter_size_g, strides=(1,1), padding='same'))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU())
        
        model.add(Conv2DTranspose(256, filter_size_g, strides=(1,1), padding='same'))
        #model.add(Conv2D(256, filter_size_g, strides=(1,1), padding='same'))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU())
        
        #model.add(Conv2DTranspose(128, filter_size_g, strides=(2,2), padding='same'))
        ##model.add(Conv2D(128, filter_size_g, strides=(1,1), padding='same'))
        #model.add(BatchNormalization(momentum=0.8))
        #model.add(LeakyReLU())
        #
        #model.add(Conv2DTranspose(64, filter_size_g, strides=(2,2), padding='same'))
        ##model.add(Conv2D(64, filter_size_g, strides=(1,1), padding='same'))
        #model.add(BatchNormalization(momentum=0.8))
        #model.add(LeakyReLU())
        
        model.add(Conv2DTranspose(32, filter_size_g, strides=(2,2), padding='same'))
        #model.add(Conv2D(32, filter_size_g, strides=(1,1), padding='same'))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU())
        
        model.add(Conv2DTranspose(16, filter_size_g, strides=(2,2), padding='same'))
        #model.add(Conv2D(16, filter_size_g, strides=(1,1), padding='same'))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU())
        
        model.add(Conv2DTranspose(8, filter_size_g, strides=(2,2), padding='same'))
        #model.add(Conv2D(8, filter_size_g, strides=(1,1), padding='same'))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU())
        
        model.add(Conv2DTranspose(img_channels, filter_size_g, strides=(1,1), padding='same'))
        model.add(Activation("tanh"))

        model.summary()

        return model

In [5]:
def build_discriminator(num_classes):

        model = Sequential()
        
        model.add(Conv2D(64, kernel_size=filter_size_d, strides = (2,2), input_shape=img_shape, padding="same"))
        #model.add(Conv2D(64, kernel_size=filter_size_d, strides = (1,1), padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        
        model.add(Conv2D(128, kernel_size=filter_size_d, strides = (2,2), padding="same"))
        #model.add(Conv2D(128, kernel_size=filter_size_d, strides = (1,1), padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        
        model.add(Conv2D(256,  kernel_size=filter_size_d, strides = (2,2), padding="same"))
        #model.add(Conv2D(256, kernel_size=filter_size_d, strides = (1,1), padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        
        model.add(Conv2D(512, kernel_size=filter_size_d, strides = (2,2), padding="same"))
        #model.add(Conv2D(512, kernel_size=filter_size_d, strides = (1,1), padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        
        #model.add(Conv2D(512, kernel_size=filter_size_d, strides = (2,2), padding="same"))
        ##model.add(Conv2D(512, kernel_size=filter_size_d, strides = (1,1), padding="same"))
        #model.add(BatchNormalization(momentum=0.8))
        #model.add(LeakyReLU(alpha=0.2))
        #model.add(Dropout(0.25))
        #
        #model.add(Conv2D(1024, kernel_size=filter_size_d, strides = (2,2), padding="same"))
        ##model.add(Conv2D(512, kernel_size=filter_size_d, strides = (1,1), padding="same"))
        #model.add(BatchNormalization(momentum=0.8))
        #model.add(LeakyReLU(alpha=0.2))
        #model.add(Dropout(0.25))
        
        model.add(Flatten())
        
        model.summary()
        
        img = Input(shape=img_shape)

        features = model(img)

        validity = Dense(1)(features)
        valid = Activation('sigmoid')(validity)
        
        label1 = Dense(1024)(features)
        lrelu1 = LeakyReLU(alpha=0.2)(label1)
        label2 = Dense(512)(label1)
        lrelu2 = LeakyReLU(alpha=0.2)(label2)
        label3 = Dense(num_classes, name = 'new_layer')(label2)
        label = Activation('softmax')(label3)
        
        return Model(img, valid), Model(img, label)

In [6]:
def generator_containing_discriminator(g, d, d_label):
    noise = Input(shape=(latent_dim,))
    img = g(noise)
    d.trainable = False
    d_label.trainable = False
    valid, target_label = d(img), d_label(img)
    
    
    return Model(noise, [valid, target_label])

In [7]:
def save_imgs(epoch, gen):
        count = 5
        noise = np.random.uniform(-1, 1, size=(count, latent_dim))
        gen_imgs = gen.predict(noise)

        gen_imgs = 127.5 * gen_imgs + 127.5
        
        if gen_imgs.shape[3] == 1:
            gen_imgs = gen_imgs[:,:,:,0]
        
        for i in range(count):
            cv2.imwrite(os.getcwd() + '\\generated\\epoch%d_%d.png' % (epoch, i), gen_imgs[i])

In [8]:
def get_images(indices, data, batch_size):
    X_train = np.zeros((batch_size, img_rows, img_cols, img_channels))
    for i in range(batch_size):
        if color_mode == 'grayscale':
            temp_img = cv2.imread(data[indices[i]], 0)
            X_train[i,:,:,0] = temp_img
        else:
            temp_img = cv2.imread(data[indices[i]])
            X_train[i] = temp_img
    X_train = (X_train - 127.5)/127.5
    return X_train

In [9]:
def get_images_classes(batch_size, data, classes):
    X_train = np.zeros((batch_size, img_rows, img_cols, img_channels))
    y_labels = np.zeros(batch_size)
    choice_arr = np.random.randint(0, len(data), batch_size)
    for i in range(batch_size):
        img_path = data[choice_arr[i]]
        temp_img = cv2.imread(img_path)
        style = img_path.split('\\')[-2]
        
        
        X_train[i] = temp_img
        y_labels[i] = classes[style]
    

    X_train = (X_train - 127.5)/127.5
    return X_train, y_labels

In [10]:
def get_images_one_class(batch_size, data, class_target):
    X_train = np.zeros((batch_size, img_rows, img_cols, img_channels))
    y_label = np.zeros(batch_size)
    '''choice_arr = np.random.randint(0, len(data[class_target]), batch_size)
    for i in range(batch_size):
        temp_img = cv2.imread(data[class_target][choice_arr[i]])

        X_train[i] = temp_img
        y_labels[i] = choice_arr[i]'''
    
    for i in range(4):
        for j in range(4):
            temp_img = cv2.imread(data[i][j])
            
            X_train[4*i + j] = temp_img
            y_label[4*i + j] = i
            
    X_train = (X_train - 127.5)/127.5
    return X_train, y_label

In [11]:
def combine_images(generated_images):
    num = generated_images.shape[0]
    width = int(math.sqrt(num))
    height = int(math.ceil(float(num)/width))
    shape = generated_images.shape[1:3]
    image = np.zeros((height*shape[0], width*shape[1], img_channels),
                     dtype=generated_images.dtype)
    for index, img in enumerate(generated_images):
        i = int(index/width)
        j = index % width
        image[i*shape[0]:(i+1)*shape[0], j*shape[1]:(j+1)*shape[1]] = \
            img[:, :, :,]
    return image

In [12]:
def get_data(missing_folders):
    styles_folder = os.listdir(path=os.getcwd() + "\\new64_images\\")
    styles_folder = [style for style in styles_folder if style not in missing_folders]
    print(styles_folder)
    classes = {}
    for i, style in enumerate(styles_folder):
        classes[style] = i
    num_styles = len(styles_folder)
    data = []
    for i in range(num_styles):
        data += glob.glob(os.getcwd() + '\\new64_images\\' + styles_folder[i] + '\\*')
    return data, num_styles, classes

In [13]:
def train_another(epochs = 100, BATCH_SIZE = 4, weights = False, month_day = '', epoch_ini = 0, missing_folders = [], 
                  gif=False):
    gif_i = 0
    if not gif:
        save_folder = 'generated'
    else:
        save_folder = 'generated_gif'
    data, num_styles, classes = get_data(missing_folders)
    
    epoch = ' ' + str(epoch_ini) + '_epoch'
    
    generator = build_generator()
    discriminator, d_label = build_discriminator(num_styles)
    
    discriminator.compile(loss=losses[0], optimizer=d_optim)
    d_label.compile(loss=losses[1], optimizer=d_optim)
    generator.compile(loss='binary_crossentropy', optimizer=g_optim)
    
    
    if month_day != '':
        generator.load_weights(os.getcwd() + '/weights/' + month_day + epoch + ' gen_weights.h5', by_name = True)
        discriminator.load_weights(os.getcwd() + '/weights/' + month_day + epoch + ' dis_weights.h5', by_name = True)
        d_label.load_weights(os.getcwd() + '/weights/' + month_day + epoch + ' dis_label_weights.h5', by_name = True)
        
    
    dcgan = generator_containing_discriminator(generator, discriminator, d_label)
    
    
    dcgan.compile(loss=losses[0], optimizer=g_optim)
    
    discriminator.trainable = True
    d_label.trainable = True
    
    
    noise = np.random.normal(0, 1, (BATCH_SIZE, latent_dim))
    for epoch in range(epoch_ini, epochs):
        if epoch % 5 == 0:
            
            
            date_today = date.today()
            
            
            
            month, day = date_today.month, date_today.day
            
            # Генерируем описание модели в формате json
            d_json = discriminator.to_json()
            # Записываем модель в файл
            json_file = open(os.getcwd() + "/%d.%d dis_model.json" % (day, month), "w")
            json_file.write(d_json)
            json_file.close()
            
            # Генерируем описание модели в формате json
            d_l_json = d_label.to_json()
            # Записываем модель в файл
            json_file = open(os.getcwd() + "/%d.%d dis_label_model.json" % (day, month), "w")
            json_file.write(d_l_json)
            json_file.close()
            
            # Генерируем описание модели в формате json
            gen_json = generator.to_json()
            # Записываем модель в файл
            json_file = open(os.getcwd() + "/%d.%d gen_model.json" % (day, month), "w")
            json_file.write(gen_json)
            json_file.close()
            
            discriminator.save_weights(os.getcwd() + '/weights/%d.%d %d_epoch dis_weights.h5' % (day, month, epoch))
            d_label.save_weights(os.getcwd() + '/weights/%d.%d %d_epoch dis_label_weights.h5' % (day, month, epoch))
            generator.save_weights(os.getcwd() + '/weights/%d.%d %d_epoch gen_weights.h5' % (day, month, epoch))
        for index in range(int(10000/BATCH_SIZE)):
            
            if not gif:
                noise = np.random.normal(0, 1, (BATCH_SIZE, latent_dim))
                
            #real_images, real_labels = get_images_classes(batch_size=BATCH_SIZE, data = data)
            
            #label = index % num_styles
            
            real_images, real_labels = get_images_classes(BATCH_SIZE, data, classes)
            
            #real_images += np.random.normal(size = img_shape, scale= 0.1)
            
            generated_images = generator.predict(noise)
            
            if index % 2 == 0:
                X = real_images
                real_labels = real_labels - 0.1 + np.random.rand(BATCH_SIZE)*0.2
                y_classif = keras.utils.to_categorical(np.zeros(BATCH_SIZE) + real_labels, num_styles)
                y = 0.8 + np.random.rand(BATCH_SIZE)*0.2
                
                d_loss = []
                d_loss.append(discriminator.train_on_batch(X, y))
                discriminator.trainable = False
                d_loss.append(d_label.train_on_batch(X, y_classif))
                print("epoch %d batch %d d_loss : %f, label_loss: %f" % (epoch, index, d_loss[0], d_loss[1]))
                print("epoch %d batch %d d_real_loss: %f" % (epoch, index, d_loss[0]))
                
                X = generated_images
                y = np.random.rand(BATCH_SIZE) * 0.2
                d_loss = discriminator.train_on_batch(X, y)
                
                print("epoch %d batch %d d_gen_loss  : %f" % (epoch, index, d_loss))
            else:
                X = generated_images
                y = np.random.rand(BATCH_SIZE) * 0.2
                d_loss = discriminator.train_on_batch(X, y)
                
                print("epoch %d batch %d d_gen_loss  : %f" % (epoch, index, d_loss))
                
                X = real_images
                real_labels = real_labels - 0.1 + np.random.rand(BATCH_SIZE)*0.2
                y_classif = keras.utils.to_categorical(np.zeros(BATCH_SIZE) + real_labels, num_styles)
                y = 0.8 + np.random.rand(BATCH_SIZE)*0.2
                d_loss = []
                d_loss.append(discriminator.train_on_batch(X, y))
                discriminator.trainable = False
                d_loss.append(d_label.train_on_batch(X, y_classif))
                
                print("epoch %d batch %d d_loss : %f, label_loss: %f" % (epoch, index, d_loss[0], d_loss[1]))
                print("epoch %d batch %d d_real_loss : %f" % (epoch, index, d_loss[0]))
            
            
            if not gif:
                noise = np.random.normal(0, 1, (BATCH_SIZE, latent_dim))
            
            discriminator.trainable = False
            d_label.trainable = False
            
            #y_classif = keras.utils.to_categorical(np.zeros(BATCH_SIZE) + 1/num_styles, num_styles)
            y_classif = np.zeros((BATCH_SIZE, num_styles))
            y_classif[:,0] += 1
            
            y = np.random.rand(BATCH_SIZE) * 0.3
            
            g_loss = dcgan.train_on_batch(noise, [y, y_classif])
            
            d_label.trainable = True
            discriminator.trainable = True
            
            print("epoch %d batch %d g_loss     : %f" % (epoch, index, g_loss[0]))
            if not gif:
                index_diviser = 50
            else:
                index_diviser = 10
            if index % index_diviser == 0:
                        image = combine_images(generated_images)
                        image = image*127.5+127.5
                        if not gif:
                            img_name = 'epoch%d_%06d.png' % (epoch, index)
                        else:
                            img_name = '%06d.png' % (gif_i)
                        cv2.imwrite(
                            os.getcwd() + '\\%s\\%s' % (save_folder, img_name), image)
                        gif_i += 1
                        #image = combine_images(real_images)
                        #image = image*127.5+127.5
                        #cv2.imwrite(
                        #    os.getcwd() + '\\generated\\epoch%d_%d_data.png' % (epoch, index), image)
                        

In [14]:
img_rows = 64
img_cols = 64
img_channels = 3
img_shape = (img_rows, img_cols, img_channels)
latent_dim = 100
filter_size_g = (5,5)
filter_size_d = (5,5)
d_strides = (2,2)

color_mode = 'rgb'

losses = ['binary_crossentropy', 'categorical_crossentropy']


#g_optim = SGD(lr = 0.0002, momentum=0.9, nesterov=True)
#d_optim = SGD(lr = 0.0002, momentum=0.9, nesterov=True)
g_optim = Adam(0.0002, beta_2 = 0.9)
d_optim = Adam(0.0002, beta_2 = 0.9)

In [15]:
train_another(1500, 64, month_day = '8.7', epoch_ini = 40,\
              missing_folders = [],
             gif = False)

['abstraktnyy-ekspressionizm', 'cognitive', 'foto', 'informel', 'kubizm', 'nyu', 'portret', 'realizm', 'suprematizm']
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 16384)             1654784   
_________________________________________________________________
batch_normalization_1 (Batch (None, 16384)             65536     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 16384)             0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 8, 8, 256)         0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 8, 8, 1024)        6554624   
_________________________________________________________________
batch_normalization_2 (Batch (None, 8, 8, 1024)        4096      
________________________

d:\programming\python36\venvs\nn\lib\site-packages\keras\engine\training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch 0 batch 0 g_loss     : 1.712866
epoch 0 batch 1 d_gen_loss  : 0.559298
epoch 0 batch 1 d_loss : 0.427126, label_loss: 4.293346
epoch 0 batch 1 d_real_loss : 0.427126
epoch 0 batch 1 g_loss     : 2.016620
epoch 0 batch 2 d_loss : 0.630598, label_loss: 5.304670
epoch 0 batch 2 d_real_loss: 0.630598
epoch 0 batch 2 d_gen_loss  : 0.955085
epoch 0 batch 2 g_loss     : 2.153728
epoch 0 batch 3 d_gen_loss  : 0.622477
epoch 0 batch 3 d_loss : 0.531876, label_loss: 4.248574
epoch 0 batch 3 d_real_loss : 0.531876
epoch 0 batch 3 g_loss     : 0.922087
epoch 0 batch 4 d_loss : 0.481415, label_loss: 4.241179
epoch 0 batch 4 d_real_loss: 0.481415
epoch 0 batch 4 d_gen_loss  : 3.259196
epoch 0 batch 4 g_loss     : 0.545195
epoch 0 batch 5 d_gen_loss  : 1.411703
epoch 0 batch 5 d_loss : 0.554700, label_loss: 4.625159
epoch 0 batch 5 d_real_loss : 0.554700
epoch 0 batch 5 g_loss     : 1.565900
epoch 0 batch 6 d_loss : 0.777630, label_loss: 4.602265
epoch 0 batch 6 d_real_loss: 0.777630
epoch 0 ba

epoch 0 batch 47 g_loss     : 2.246449
epoch 0 batch 48 d_loss : 0.365371, label_loss: 3.209774
epoch 0 batch 48 d_real_loss: 0.365371
epoch 0 batch 48 d_gen_loss  : 1.701092
epoch 0 batch 48 g_loss     : 2.305713
epoch 0 batch 49 d_gen_loss  : 1.371297
epoch 0 batch 49 d_loss : 0.681586, label_loss: 3.003491
epoch 0 batch 49 d_real_loss : 0.681586
epoch 0 batch 49 g_loss     : 2.590014
epoch 0 batch 50 d_loss : 0.462484, label_loss: 3.116238
epoch 0 batch 50 d_real_loss: 0.462484
epoch 0 batch 50 d_gen_loss  : 1.611036
epoch 0 batch 50 g_loss     : 2.728052
epoch 0 batch 51 d_gen_loss  : 1.784187
epoch 0 batch 51 d_loss : 0.365929, label_loss: 3.653413
epoch 0 batch 51 d_real_loss : 0.365929
epoch 0 batch 51 g_loss     : 2.275364
epoch 0 batch 52 d_loss : 0.386789, label_loss: 3.429457
epoch 0 batch 52 d_real_loss: 0.386789
epoch 0 batch 52 d_gen_loss  : 1.573586
epoch 0 batch 52 g_loss     : 2.131648
epoch 0 batch 53 d_gen_loss  : 1.558842
epoch 0 batch 53 d_loss : 0.466432, label_lo

epoch 0 batch 94 g_loss     : 0.914340
epoch 0 batch 95 d_gen_loss  : 0.354024
epoch 0 batch 95 d_loss : 0.343522, label_loss: 3.078123
epoch 0 batch 95 d_real_loss : 0.343522
epoch 0 batch 95 g_loss     : 0.969206
epoch 0 batch 96 d_loss : 0.366106, label_loss: 3.602901
epoch 0 batch 96 d_real_loss: 0.366106
epoch 0 batch 96 d_gen_loss  : 0.353162
epoch 0 batch 96 g_loss     : 0.962381
epoch 0 batch 97 d_gen_loss  : 0.356336
epoch 0 batch 97 d_loss : 0.316221, label_loss: 2.492940
epoch 0 batch 97 d_real_loss : 0.316221
epoch 0 batch 97 g_loss     : 0.892631
epoch 0 batch 98 d_loss : 0.335908, label_loss: 2.478072
epoch 0 batch 98 d_real_loss: 0.335908
epoch 0 batch 98 d_gen_loss  : 0.387927
epoch 0 batch 98 g_loss     : 0.887905
epoch 0 batch 99 d_gen_loss  : 0.386401
epoch 0 batch 99 d_loss : 0.316884, label_loss: 2.549985
epoch 0 batch 99 d_real_loss : 0.316884
epoch 0 batch 99 g_loss     : 0.894451
epoch 0 batch 100 d_loss : 0.316230, label_loss: 2.464099
epoch 0 batch 100 d_real_

...

epoch 39 batch 119 d_gen_loss  : 0.341222
epoch 39 batch 119 d_loss : 0.331009, label_loss: 1.473180
epoch 39 batch 119 d_real_loss : 0.331009
epoch 39 batch 119 g_loss     : 0.552978
epoch 39 batch 120 d_loss : 0.352007, label_loss: 1.398741
epoch 39 batch 120 d_real_loss: 0.352007
epoch 39 batch 120 d_gen_loss  : 0.319069
epoch 39 batch 120 g_loss     : 0.559535
epoch 39 batch 121 d_gen_loss  : 0.311621
epoch 39 batch 121 d_loss : 0.363198, label_loss: 1.611694
epoch 39 batch 121 d_real_loss : 0.363198
epoch 39 batch 121 g_loss     : 0.573426
epoch 39 batch 122 d_loss : 0.357403, label_loss: 1.485250
epoch 39 batch 122 d_real_loss: 0.357403
epoch 39 batch 122 d_gen_loss  : 0.338549
epoch 39 batch 122 g_loss     : 0.523793
epoch 39 batch 123 d_gen_loss  : 0.315621
epoch 39 batch 123 d_loss : 0.318208, label_loss: 1.646018
epoch 39 batch 123 d_real_loss : 0.318208
epoch 39 batch 123 g_loss     : 0.543605
epoch 39 batch 124 d_loss : 0.343424, label_loss: 1.571966
epoch 39 batch 124 d_re

epoch 40 batch 8 d_loss : 0.360364, label_loss: 1.266364
epoch 40 batch 8 d_real_loss: 0.360364
epoch 40 batch 8 d_gen_loss  : 0.367897
epoch 40 batch 8 g_loss     : 0.537309
epoch 40 batch 9 d_gen_loss  : 0.334926
epoch 40 batch 9 d_loss : 0.324512, label_loss: 1.430977
epoch 40 batch 9 d_real_loss : 0.324512
epoch 40 batch 9 g_loss     : 0.518129
epoch 40 batch 10 d_loss : 0.328436, label_loss: 1.355752
epoch 40 batch 10 d_real_loss: 0.328436
epoch 40 batch 10 d_gen_loss  : 0.352836
epoch 40 batch 10 g_loss     : 0.498392
epoch 40 batch 11 d_gen_loss  : 0.351719
epoch 40 batch 11 d_loss : 0.344198, label_loss: 1.375211
epoch 40 batch 11 d_real_loss : 0.344198
epoch 40 batch 11 g_loss     : 0.553340
epoch 40 batch 12 d_loss : 0.339701, label_loss: 1.496787
epoch 40 batch 12 d_real_loss: 0.339701
epoch 40 batch 12 d_gen_loss  : 0.335388
epoch 40 batch 12 g_loss     : 0.487011
epoch 40 batch 13 d_gen_loss  : 0.362953
epoch 40 batch 13 d_loss : 0.325508, label_loss: 1.586368
epoch 40 bat

epoch 40 batch 53 g_loss     : 0.554083
epoch 40 batch 54 d_loss : 0.347581, label_loss: 1.460912
epoch 40 batch 54 d_real_loss: 0.347581
epoch 40 batch 54 d_gen_loss  : 0.325109
epoch 40 batch 54 g_loss     : 0.538997
epoch 40 batch 55 d_gen_loss  : 0.344238
epoch 40 batch 55 d_loss : 0.343759, label_loss: 1.393095
epoch 40 batch 55 d_real_loss : 0.343759
epoch 40 batch 55 g_loss     : 0.544093
epoch 40 batch 56 d_loss : 0.319323, label_loss: 1.577812
epoch 40 batch 56 d_real_loss: 0.319323
epoch 40 batch 56 d_gen_loss  : 0.329764
epoch 40 batch 56 g_loss     : 0.552061
epoch 40 batch 57 d_gen_loss  : 0.345944
epoch 40 batch 57 d_loss : 0.327906, label_loss: 1.741884
epoch 40 batch 57 d_real_loss : 0.327906
epoch 40 batch 57 g_loss     : 0.493692
epoch 40 batch 58 d_loss : 0.323445, label_loss: 1.594393
epoch 40 batch 58 d_real_loss: 0.323445
epoch 40 batch 58 d_gen_loss  : 0.337235
epoch 40 batch 58 g_loss     : 0.512567
epoch 40 batch 59 d_gen_loss  : 0.332096
epoch 40 batch 59 d_lo

epoch 40 batch 99 g_loss     : 0.545976
epoch 40 batch 100 d_loss : 0.332620, label_loss: 1.382022
epoch 40 batch 100 d_real_loss: 0.332620
epoch 40 batch 100 d_gen_loss  : 0.333325
epoch 40 batch 100 g_loss     : 0.533991
epoch 40 batch 101 d_gen_loss  : 0.345811
epoch 40 batch 101 d_loss : 0.370171, label_loss: 1.542296
epoch 40 batch 101 d_real_loss : 0.370171
epoch 40 batch 101 g_loss     : 0.480294
epoch 40 batch 102 d_loss : 0.339086, label_loss: 1.749621
epoch 40 batch 102 d_real_loss: 0.339086
epoch 40 batch 102 d_gen_loss  : 0.354983
epoch 40 batch 102 g_loss     : 0.504135
epoch 40 batch 103 d_gen_loss  : 0.338237
epoch 40 batch 103 d_loss : 0.321109, label_loss: 1.530406
epoch 40 batch 103 d_real_loss : 0.321109
epoch 40 batch 103 g_loss     : 0.497050
epoch 40 batch 104 d_loss : 0.342455, label_loss: 1.673231
epoch 40 batch 104 d_real_loss: 0.342455
epoch 40 batch 104 d_gen_loss  : 0.351289
epoch 40 batch 104 g_loss     : 0.539679
epoch 40 batch 105 d_gen_loss  : 0.327617
e

KeyboardInterrupt: 

In [ ]:
class art_creator:
    def __init__(self):
        self.generator = self.build_generator()
        self.extractor = self.build_extractor()
    

import tensorflow as tf
tf.test.is_gpu_available()